In [1]:
from ReadData import read_data
import xpress as xp
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from ReadData import interp

df_RE, df_Grid, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods= read_data()

In [ ]:
def model_v1(df_RE, df_Intensity, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods, miprelstop = 0.02, maxtime = 120):
    '''
    Solves hydrogen production problem for Wood Mackenzie optimising costs and emissions

    Inputs:
        df_RE: DataFrame with hourly solar, offshore wind, onshore wind data
        df_Grid: DataFrame with hourly CO2 intensity of power from grid and its price
        df_Elctro: DataFrame with Electrolyser parameters for PEM ALK SOEC 
        df_Elctro_Costs: DataFrame with CAPEX costs of electrolysers based on different capacities
        df_Battery: DataFrame with Battery storage costs
        df_repperiods: DataFrame with representative periods and their respective beginning and end indexes and weights
        df_PPA: DataFrame with PPA prices for Solar, Wind onshore and offshore
        miprelstop: MIP gap at which to terminate (default at 2%)
        maxtime: max time to allow the solver to run for (default 120s)
    
    Outputs:

    '''

    # Notes of things to change:
    # 1. Change stack replacement cost to changing over years and dependant on the year ur replacing it in
    # 2. Change PPA to be variable
    # 3. Add variable efficiencies of electrolyser
    # 4. Change to correct units
    # 5. Objective: CAPEX Costs: PPA + Battery Capacity + H2 Store capacity + Electrolyser Capacity\
                #   OPEX Costs: Stack replacement dependant on diff lifetimes and prices + Power costs for 25 yrs based on rep periods + Fixed opex for battery, store, electrolyser

    xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

    def clean_name(r):
        return r.replace(' ', '_')

    prob = xp.problem(name="Hydrogen WoodMac")


    # ------------ SETS -------------
    T = df_RE.index.unique()
    E = list(df_Elctro['Type'].unique())
    R = {col for col in df_RE.columns if not col.startswith('Report_')}
    K = df_repperiods['K'].index

    # ------ DECISION VARIABLES ------

    # Proportion of renewable energy contracted to take from renewable site r
    PPA = {(r): xp.var(vartype=xp.continuous, name = f'PPA_{clean_name(r)}', lb= 0, ub=1) for r in R}

    # Power bought from the grid at time t (kW)
    P_Grid_b = {(t): xp.var(vartype=xp.continuous, name = f'P_Grid_b_{t}') for t in T}

    # Power sold to the grid at time t (kW)
    P_Grid_s = {(t): xp.var(vartype=xp.continuous, name = f'P_Grid_s_{t}') for t in T}

    # Power taken out of battery at time t (kW)
    P_Bat_out = {(t): xp.var(vartype=xp.continuous, name = f'P_Bat_out_{t}') for t in T}

    # Power put into battery at time t (kW)
    P_Bat_in = {(t): xp.var(vartype=xp.continuous, name = f'P_Bat_in_{t}') for t in T}

    # Power put into electrolyser e at time t (kW)
    P_Ez = {(e, t): xp.var(vartype=xp.continuous, name = f'P_Ez_{e}_{t}') for e in E for t in T}

    # Power required for putting H2 into storage at time t (kW)
    P_H2st = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_{t}') for t in T}

    # Hydrogen leaving store at time t (kg/h)
    H_H2st_out = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_out_{t}') for t in T}

    # Hydrogen entering store at time t (kg/h)
    H_H2st_in = {(t): xp.var(vartype=xp.continuous, name = f'P_H2st_in_{t}') for t in T}

    #  Hydrogen leaving electrolyser e at time t (kg/h)
    H_Ez_out = {(e, t): xp.var(vartype=xp.continuous, name = f'H_Ez_out_{e}_{t}') for e in E for t in T}

    # Energy stored in battery at time t (kWh)
    E_Bat = {(t): xp.var(vartype=xp.continuous, name = f'E_Bat_{t}') for t in T}

    # Hydrogen stored at time t (kg)
    E_H2st = {(t): xp.var(vartype=xp.continuous, name = f'E_H2st_{t}') for t in T}

    # Energy Capacity of battery (kWh)
    Q_Bat_cap = xp.var(vartype=xp.continuous, name='Q_Bat_cap')

    # Energy capacity of H2 storage tank (kg)
    Q_H2st_cap = xp.var(vartype=xp.continuous, name='Q_H2st_cap')

    # Power capacity of electrolyser e (kW)
    Q_Ez_cap = {(e): xp.var(vartype=xp.continuous, name = f'Q_Ez_cap_{e}') for e in E}

    # --------- PARAMETERS ------------

    # Electrolysers:

    # Change Elctro df to be indexed by electrolyser name by making new DatFrame df_Elctro_index
    df_Elctro_index = df_Elctro.copy()
    df_Elctro_index.set_index('Type', inplace=True)

    # Data from electrolyser parameters
    Ez_kWh_per_kg = df_Elctro_index['kWh/kg H2'].to_dict()
    Ez_kWh_per_kg = df_Elctro_index['kWh/kg H2'].to_dict()
    Ez_min_load = df_Elctro_index['Minimum Load'].to_dict()
    Ez_max_load = df_Elctro_index['Maximum Load'].to_dict()
    # THE BELOW 1 NEED TO BE TAKEN OUT WHEN I CONSIDER ACTUAL STACK LIFE 
    Ez_stack_life = df_Elctro_index['Stack Lifetime (hours)'].to_dict()
    Ez_deg_rate = df_Elctro_index['Efficiency degradation / year'].to_dict()
    Ez_fixed_opex = df_Elctro_index['Fixed Opex percent'].to_dict()

    # Electrolyser Costs & Interpolation:

    # Initialise dictionaries for the interpolated values
    Ez_Total_CAPEX_interp ={}
    Ez_Stack_Replacement_interp ={}

    # For each electrolyser create the interpolated values
    for e in E:
        df_interpolate = df_Elctro_Costs[df_Elctro_Costs['Technology'] == e]

        # Total CAPEX costs for each electrolyser
        Ez_Total_CAPEX_interp[e] = interp1d(
            df_interpolate['Scale (kW)'],
            df_interpolate['Total Installed Cost (TIC) (£/kW)'],
            kind = 'linear',
            fill_value='extrapolate',
        )

        # Stack replacement costs for each electrolyser
        #  THE BELOW NEEDS TO BE ADJUSTED WHEN I CONSIDER STACK AS CHANGING OVER 
        Ez_Stack_Replacement_interp[e] = interp1d(
            df_interpolate['Scale (kW)'],
            df_interpolate['Stack cappex'],
            kind = 'linear',
            fill_value='extrapolate',
        )
    # Use interp function to get values for the two parametrs above

    # PPA Costs:
    # Change df_PPA to df_PPA_index by indexing by renewable source name
    df_PPA_index = df_PPA.copy()
    df_PPA_index.set_index('Renewable Source', inplace=True)
    C_PPA = df_PPA_index['PPA Price (£/kWh)']
    
    # Representative Periods:

    # Grid Costs and Intensity:
    C_Grid = df_Grid['Price (£/kWh, real 2025)'].to_dict()
    Int_Grid = df_Grid['CO2 Intensity (kg CO2/kWh)'].to_dict()
    
    # Renewable Energy:
    # to get a value index by time period as t = row index and renewable source r: x = df_RE.loc[t,r]
    P_PPA = {(r, t): df_RE.loc[t, r] for t in df_RE.index for r in df_RE.columns}

    # ADD DEMAND!!!!
    D_H2 = 5000


    # ---------- CONSTRAINTS ------------

    # Power Balance:
    prob.addConstraint( xp.Sum(PPA[r]*P_PPA[r,t] for r in R )+ P_Grid_b[t]+ P_Bat_out[t] == P_Grid_s[t] +P_Bat_in[t] + P_H2st[t] + xp.Sum(P_Ez[e,t] for e in E) for t in T)
    
    # Hydrogen Balance:
    prob.addConstraint( H_H2st_out[t] + xp.Sum(H_Ez_out[e,t] for e in E) == H_H2st_in[t] + D_H2 for t in T)

    # H2 Storage:

    # Electrolyser:

    # Battery:

    # Average CO2 Emissions:

    # Unit Commitment:

    # Stack Replacement:

    # ---------- OBJECTIVE FUNCTION ----------

    # CAPEX Costs:
    # Electrolyser + H2 Storage + Battery based on Capacities
    # capacities = xp.Sum(xp.Sum(PPA[r]*P_PPA[r,t]*C_PPA[r] for r in R) + C_ for t in T)
    # depends on capacity chosen!!!

    # OPEX Costs:
    # Electrolyser + H2 Storage + Battery fixed OPEX based on % of CAPEX

    # Variable OPEX for Power bought and sold on Grid

    # Stack Replacement Costs


    return T,E,R,K,PPA

T,E,R,K,PPA = model_v1(df_RE, df_Grid, df_Elctro, df_Elctro_Costs, df_Battery, df_PPA, df_repperiods, miprelstop = 0.02, maxtime = 120)
print(T,E,R,K)
print(PPA)

RangeIndex(start=0, stop=227904, step=1) ['PEMWE', 'AWE', 'SOEC'] {'Solar PV', 'Wind Onshore', 'Wind Offshore'} RangeIndex(start=0, stop=12, step=1)
Renewable Source
Solar PV         0.0490
Wind Onshore     0.0456
Wind Offshore    0.0422
Name: PPA Price (£/kWh), dtype: float64
